Coding out the MNIST digit recognition neural network using only numpy. From this youtube video: https://www.youtube.com/watch?v=w8yWXqWQ%20YmU&ab_channel=SamsonZhang

Just as a way to practice and fully understand what's happening with basic neural network architecture.

~Samyukt Sriram

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#Loading in the data, manually doing train test split.

data = pd.read_csv('../input/digit-recognizer/train.csv')
data = np.array(data)

In [ ]:
m , n = data.shape #n is number of features + 1, the label is present in first row
np.random.shuffle(data)

data_dev = data[0:1000].T #Transposed so each example is a column. Not clear on why this is important. Easier for indexing maybe?
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255

In [ ]:
def init_params():
    
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10,1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10,1) - 0.5
    
    print(W1, b1, W2, b2)
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    a = np.exp(Z) / sum(np.exp(Z))
    return a

def one_hot(Y):
    
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def deriv_ReLU(Z):
    return Z > 0

def forward_prop(W1, b1, W2, b2, X): #X is input
    
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    
    return Z1, A1, Z2, A2

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    
    m = Y.size
    one_hot_Y = one_hot(Y)
    
    dZ2 = A2 - one_hot_Y #Is this Cost?
    
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * deriv_ReLU(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1
    W2 = W2 - alpha * dW2
    b2 = b2 - alpha * db2
    
    return W1, b1, W2, b2

def get_predictions(A2):
    return np.argmax(A2, 0) #np.argmax returns an index, but we happen to want 0 to 9 anyway so this works out

def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

def gradient_descent(X, Y, iterations, alpha):
    
    #Init params
    #for loop over iterations: forward_prop -> back_prop -> update params
    #save params.
    
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):

        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = back_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        if i % 50 ==0:
            print(f'Iteration: {i} \n Accuracy:{get_accuracy(get_predictions(A2), Y)}')
    return W1, b1, W2, b2

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 400, 0.1)

print(W1, b1, W2, b2)